In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv
import sqlite3
import timeit
start_time = timeit.default_timer()

# Data Collection For Team Table

In [2]:
#create the chromedriver connection
browser = Browser('chrome')

In [3]:
#specify the webiste and use BeautifulSoup to create the Parser
teams_url = 'https://www.spotrac.com/nfl/cap/'
browser.visit(teams_url)
html1 = browser.html
soup = soup(html1, 'html.parser')

In [4]:
#loop through and replace all the 'span'elemnts as they create double strings later on
for span_tag in soup.find_all('span', style='display:none'):
    span_tag.replace_with('')

In [5]:
#Create a list for the end syntax of each url so we can loop through specific teams
url_end = []
#delete the leauge average row
for average in soup.find_all('tr', class_='average'):
        average.extract()
#Loop through extract team names and formate to match urls ending format
for team in soup.find_all('span', class_='xs-hide'):
    format = team.string.replace(" ", "-")+'/cap/'
    url_end.append(format)
    

In [6]:
#Create list of team names that we will use for the table_names of each teams data
table_names = []
for average in soup.find_all('tr', class_='average'):
        average.extract()
for team2 in soup.find_all('span', class_='xs-hide'):
    format2 = team2.string.replace(" ", "_")
    table_names.append(format2)

In [7]:
table_names

['Pittsburgh_Steelers',
 'New_York_Jets',
 'Tennessee_Titans',
 'Chicago_Bears',
 'Atlanta_Falcons',
 'Arizona_Cardinals',
 'New_England_Patriots',
 'Cincinnati_Bengals',
 'Las_Vegas_Raiders',
 'Los_Angeles_Chargers',
 'Carolina_Panthers',
 'Green_Bay_Packers',
 'Cleveland_Browns',
 'Detroit_Lions',
 'Houston_Texans',
 'Washington_Commanders',
 'Indianapolis_Colts',
 'Miami_Dolphins',
 'Seattle_Seahawks',
 'San_Francisco_49ers',
 'Buffalo_Bills',
 'Denver_Broncos',
 'Baltimore_Ravens',
 'Philadelphia_Eagles',
 'Jacksonville_Jaguars',
 'Los_Angeles_Rams',
 'Kansas_City_Chiefs',
 'Minnesota_Vikings',
 'Dallas_Cowboys',
 'New_York_Giants',
 'New_Orleans_Saints',
 'Tampa_Bay_Buccaneers']

In [8]:
url_end

['Pittsburgh-Steelers/cap/',
 'New-York-Jets/cap/',
 'Tennessee-Titans/cap/',
 'Chicago-Bears/cap/',
 'Atlanta-Falcons/cap/',
 'Arizona-Cardinals/cap/',
 'New-England-Patriots/cap/',
 'Cincinnati-Bengals/cap/',
 'Las-Vegas-Raiders/cap/',
 'Los-Angeles-Chargers/cap/',
 'Carolina-Panthers/cap/',
 'Green-Bay-Packers/cap/',
 'Cleveland-Browns/cap/',
 'Detroit-Lions/cap/',
 'Houston-Texans/cap/',
 'Washington-Commanders/cap/',
 'Indianapolis-Colts/cap/',
 'Miami-Dolphins/cap/',
 'Seattle-Seahawks/cap/',
 'San-Francisco-49ers/cap/',
 'Buffalo-Bills/cap/',
 'Denver-Broncos/cap/',
 'Baltimore-Ravens/cap/',
 'Philadelphia-Eagles/cap/',
 'Jacksonville-Jaguars/cap/',
 'Los-Angeles-Rams/cap/',
 'Kansas-City-Chiefs/cap/',
 'Minnesota-Vikings/cap/',
 'Dallas-Cowboys/cap/',
 'New-York-Giants/cap/',
 'New-Orleans-Saints/cap/',
 'Tampa-Bay-Buccaneers/cap/']

In [9]:
#scrape the website to find elements within the table
cap = soup.find_all('td')

In [10]:
#create the distribution by looping through the element
distribution1 = [value.text.strip() for value in cap]

#Print only the team names and not the \n abrreviation
distribution1 = [s.split('\n')[0] for s in distribution1]


In [11]:
#print the distribution to check everything prints right
distribution1[0::][::10][0]

'1'

In [12]:
#Create column names for our data
column_names = [
    "Team_id"
    "Teams", 
    "Position", 
    "Signed", 
    "Avg_Age", 
    "Active", 
    "Dead_Cap", 
    "Top_51_Cap", 
    "Cap_Space_(Top 51)", 
    "Total_Cap", 
    "Cap_Space(All)"
]

In [13]:
#Create Dictionary to store the list data
team_dictionary = {}

team_dictionary["Team_id"] = np.arange(1, len(distribution1[0::][::10])+1)
team_dictionary["Teams"] = distribution1[1::][::10]
team_dictionary["Rank"] = distribution1[0::][::10]
team_dictionary["Signed"] = distribution1[2::][::10]
team_dictionary["Avg_Age"] = distribution1[3::][::10]
team_dictionary["Active"] = distribution1[4::][::10]
team_dictionary["Dead_Cap"] = distribution1[5::][::10]
team_dictionary["Top_51_Cap"] = distribution1[6::][::10]
team_dictionary["Cap_Space_Top_51"] = distribution1[7::][::10]
team_dictionary["Total_Cap"] = distribution1[8::][::10]
team_dictionary["Cap_Space_All"] = distribution1[9::][::10]
    
    
#convert that dictionary to a DataFrame so its easier to manage    
team_df = pd.DataFrame(data=team_dictionary)



In [14]:
team_df

,Team_id,Teams,Rank,Signed,Avg_Age,Active,Dead_Cap,Top_51_Cap,Cap_Space_Top_51,Total_Cap,Cap_Space_All
0,1,Pittsburgh Steelers,1,15,28.47,"$166,625,837","$4,755,000","$171,380,837","$57,846,308","$171,380,837","$57,846,308"
1,2,New York Jets,2,21,27.19,"$171,997,744",-,"$171,997,744","$56,582,966","$171,997,744","$56,582,966"
2,3,Tennessee Titans,3,19,26.63,"$145,054,659","$30,936,668","$175,991,327","$54,907,707","$175,991,327","$54,907,707"
3,4,Chicago Bears,4,58,26.16,"$168,820,850","$22,968,810","$187,289,660","$45,149,196","$191,789,660","$40,649,196"
4,5,Atlanta Falcons,5,67,26.31,"$182,103,124","$16,585,999","$189,569,123","$40,802,422","$198,689,123","$31,682,422"
5,6,Arizona Cardinals,6,53,26.85,"$185,081,444","$11,562,966","$196,644,410","$32,655,131","$196,644,410","$32,655,131"
6,7,New England Patriots,7,62,27.52,"$182,639,828","$16,559,604","$192,209,432","$30,170,816","$199,199,432","$23,180,816"
7,8,Cincinnati Bengals,8,57,26.05,"$200,461,052","$619,060","$196,580,112","$30,028,624","$201,080,112","$25,528,624"
8,9,Las Vegas Raiders,9,56,26.46,"$178,718,845","$30,062,498","$207,281,343","$27,719,925","$208,781,343","$26,219,925"
9,10,Los Angeles Chargers,10,56,26.38,"$202,339,152","$2,185,926","$202,275,078","$24,245,886","$204,525,078","$21,995,886"


In [15]:
#close out of chromedriver browser
browser.quit()

In [16]:
# create a connection to an SQLite database
with sqlite3.connect('sport_caps.db') as con:

# write the dataframe to an SQLite table
    team_df.to_sql('nfl_teams', con=con, if_exists='replace', index=False, dtype={'Team_id': 'INTEGER PRIMARY KEY AUTOINCREMENT'})

con.commit()
# close the database connection
con.close()

# Data Collection For Individual Teams and Players

In [17]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Create the list of urls using the base and endings that we defined ad url_end
full_urls = []
base_url = 'https://www.spotrac.com/nfl/'
for front_url in url_end:
    full_url = (base_url + front_url)
    full_urls.append(full_url)
    
full_urls

['https://www.spotrac.com/nfl/Pittsburgh-Steelers/cap/',
 'https://www.spotrac.com/nfl/New-York-Jets/cap/',
 'https://www.spotrac.com/nfl/Tennessee-Titans/cap/',
 'https://www.spotrac.com/nfl/Chicago-Bears/cap/',
 'https://www.spotrac.com/nfl/Atlanta-Falcons/cap/',
 'https://www.spotrac.com/nfl/Arizona-Cardinals/cap/',
 'https://www.spotrac.com/nfl/New-England-Patriots/cap/',
 'https://www.spotrac.com/nfl/Cincinnati-Bengals/cap/',
 'https://www.spotrac.com/nfl/Las-Vegas-Raiders/cap/',
 'https://www.spotrac.com/nfl/Los-Angeles-Chargers/cap/',
 'https://www.spotrac.com/nfl/Carolina-Panthers/cap/',
 'https://www.spotrac.com/nfl/Green-Bay-Packers/cap/',
 'https://www.spotrac.com/nfl/Cleveland-Browns/cap/',
 'https://www.spotrac.com/nfl/Detroit-Lions/cap/',
 'https://www.spotrac.com/nfl/Houston-Texans/cap/',
 'https://www.spotrac.com/nfl/Washington-Commanders/cap/',
 'https://www.spotrac.com/nfl/Indianapolis-Colts/cap/',
 'https://www.spotrac.com/nfl/Miami-Dolphins/cap/',
 'https://www.spot

In [18]:
import timeit
start_time = timeit.default_timer()
# empty lists to hold dataframes and counter variables 
dfs = []
player_dfs = []
player_id = 1
team_id = 0


# create a new browser instance
with Browser('chrome') as browser:
    for url in full_urls:
        # visit the URL in the browser
        browser.visit(url)

        # get the HTML content of the page
        html = browser.html

        # parse the HTML content using Beautiful Soup
        soup = BeautifulSoup(html, 'html.parser')

        # extract the data you want and convert it into a dataframe
        for span_tag in soup.find_all('span', style='display:none'):
            span_tag.replace_with('')
        table = soup.find('tbody')
        distribution = [value.text for value in table.find_all('tr', role="row")]
        new_distribution = []
        for s in distribution:
            new_distribution += s.split('\n')
            final_distribution = [item for item in new_distribution if item != '']

        
        
    #Create Dictionary to store the list data
        dictionary = {
            "Team_id": distribution1[0::][::10][team_id],
            "Player_id": np.arange(player_id, player_id+len(final_distribution[0::13])),
            "Active_Players": final_distribution[0::13],
            "Position": final_distribution[1::13],
            "Base_Salary": final_distribution[3::13],
            "Signing_Bonus": final_distribution[4::13],
            "Roster_Bonus": final_distribution[5::13],
            "Option_Bonus": final_distribution[6::13],
            "Workout_Bonus": final_distribution[7::13],
            "Restruc_Bonus": final_distribution[8::13],
            "Misc": final_distribution[9::13],
            "Dead_Cap": final_distribution[10::13],
            "Cap_Hit": final_distribution[2::13],
            "Cap_Percentage": final_distribution[12::13]
        }

        
        
    #Create DataFrame from the dictionary
        df = pd.DataFrame(data=dictionary)
    
    #Append the dataframe to the list
        dfs.append(df)
    
    #For Players Table Create New Dictionary and add the data
        active_players = {
            "Player_id": np.arange(player_id, player_id + len(final_distribution[0::13])),
            "Team_id": distribution1[0::][::10][team_id],
            "Active_Players": final_distribution[0::13],
            "Position": final_distribution[1::13],
            "Base_Salary": final_distribution[3::13],
            "Cap_Hit": final_distribution[2::13],
            "Cap_Percentage": final_distribution[12::13]
        }

    
    #Append the dataframe to the list
        player_df = pd.DataFrame(data=active_players,index=None)
        player_dfs.append(player_df)
    #Increment the player_id and team_id for the next iteration
        player_id = player_id + len(final_distribution[0::][::13])
        team_id = team_id + 1

#Combine the Dataframe into one singular one        
full_player_df = pd.concat(player_dfs)
end_time = timeit.default_timer()
print("Time taken: ", end_time - start_time, " seconds.")

Time taken:  137.380981958  seconds.


In [19]:
dfs[29]

,Team_id,Player_id,Active_Players,Position,Base_Salary,Signing_Bonus,Roster_Bonus,Option_Bonus,Workout_Bonus,Restruc_Bonus,Misc,Dead_Cap,Cap_Hit,Cap_Percentage
0,30,1678,Leonard Williams,DE,"$18,000,000","$7,500,000",-,-,-,"$6,760,000",-,"($20,220,000)","$32,260,000",14.05
1,30,1679,Kenny Golladay,WR,"$13,250,000","$3,400,000","$4,500,000",-,"$250,000",-,-,"($14,700,000)","$21,400,000",9.32
2,30,1680,Daniel Jones,QB,"$9,500,000","$9,000,000",-,-,"$500,000",-,"$2,000,000","($82,000,000)","$21,000,000",9.14
3,30,1681,Adoree' Jackson,CB,"$11,000,000","$4,500,000","$588,230",-,-,"$2,988,333",-,"($10,476,667)","$19,076,563",8.31
4,30,1682,Dexter Lawrence,DT,"$12,407,000",-,-,-,-,-,-,"($10,753,000)","$12,407,000",5.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,30,1738,Kalil Pimpleton,WR,"$750,000",-,-,-,-,-,-,-,"$750,000",0.33
61,30,1739,Makai Polk,WR,"$750,000",-,-,-,-,-,-,-,"$750,000",0.33
62,30,1740,Trenton Thompson,S,"$750,000",-,-,-,-,-,-,-,"$750,000",0.33
63,30,1741,Jamie Gillan,P,-,-,-,-,-,-,-,-,-,0.00


In [20]:
#close out of chromedriver browser
browser.quit()

In [21]:
# create a connection to the database
# important to create the .db file first in sqlite before running this code as some schema error can occur if not
conn = sqlite3.connect('sport_caps.db')

# iterate over each dataframe and table name, and write each dataframe to the corresponding table
for df, table_name in zip(dfs, table_names):
    df.to_sql(table_name, conn, if_exists='replace', index=False, dtype={'Player_id': 'INTEGER PRIMARY KEY AUTOINCREMENT', 'Team_id': 'INTEGER'})

conn.commit()

# close the database connection
conn.close()


In [22]:
#check to make sure full player list dataframe is corerct 
full_player_df

,Player_id,Team_id,Active_Players,Position,Base_Salary,Cap_Hit,Cap_Percentage
0,1,1,T.J. Watt,OLB,"$20,000,000","$29,368,694",12.81
1,2,1,Cameron Heyward,DE,"$15,850,000","$22,256,250",9.71
2,3,1,Minkah Fitzpatrick,S,"$14,500,000","$18,000,000",7.85
3,4,1,Diontae Johnson,WR,"$8,000,000","$16,333,333",7.13
4,5,1,Chukwuma Okorafor,RT,"$6,000,000","$13,083,333",5.71
...,...,...,...,...,...,...,...
49,1850,32,Willington Prevlion,DT,"$750,000","$750,000",0.33
50,1851,32,Hamilcar Rashed,DE,"$750,000","$750,000",0.33
51,1852,32,Charles Snowden,DE,"$750,000","$750,000",0.33
52,1853,32,Jake Verity,K,"$750,000","$750,000",0.33


In [23]:
conn = sqlite3.connect('sport_caps.db')

# create player table in nfl_cap database

full_player_df.to_sql('Players', conn, if_exists='replace', index=False, dtype={'Player_id': 'INTEGER PRIMARY KEY AUTOINCREMENT'})

conn.commit()

# close the database connection
conn.close()

In [24]:
end_time = timeit.default_timer()
print("Time taken: ", end_time - start_time, " seconds.")

Time taken:  205.85312933299997  seconds.
